In [4]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder


In [5]:
current_dir = os.getcwd()

# Original Dataset 
filename = 'df_processed.csv'
filepath = os.path.normpath(os.path.join(current_dir, '../data/processed/', filename))


chunk_size = 10000
chunks = []

for chunk in pd.read_csv(filepath, chunksize=chunk_size):
    chunk.drop_duplicates(inplace=True)
    chunk.dropna(inplace=True)
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

In [31]:
df_users = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../data/features/', 'users_behaviur.csv')))
df_words = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../data/features/', 'word_complexity_features.csv')), sep='\t')

In [32]:
df.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,history_seen,history_correct,session_seen,session_correct,h_recall,lang_combination
0,1.000000,1362082032,444407,u:dDwF,es,en,73eecb492ca758ddab5371cf7b5cca32,3,3,1,1,1.000000,en-es
1,1.000000,1362082044,5963,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,8,6,6,6,0.750000,en-de
2,0.750000,1362082044,5963,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,6,5,4,3,0.833333,en-de
3,0.888889,1362082044,5963,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,6,5,9,8,0.833333,en-de
4,0.800000,1362082044,5963,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,8,6,5,4,0.750000,en-de


In [33]:
df_1 = df.merge(df_words, on = 'lexeme_id', how='inner')

In [34]:
df_1

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,history_seen,history_correct,session_seen,...,gender,def,tense,POS,person,number,word,word_len,tags_list,SUBTLEX
0,1.000000,1362082032,444407,u:dDwF,es,en,73eecb492ca758ddab5371cf7b5cca32,3,3,1,...,NaN,NaN,NaN,pr,NaN,NaN,bajo,4,['pr'],111241.0
1,1.000000,1362082044,5963,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,8,6,6,...,NaN,NaN,pri,vblex,p3,sg,lernt,5,"['vblex', 'pri', 'p3', 'sg']",3391.0
2,0.750000,1362082044,5963,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,6,5,4,...,f,def,NaN,det,NaN,sg,die,3,"['det', 'def', 'f', 'sg']",2484854.0
3,0.888889,1362082044,5963,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,6,5,9,...,m,NaN,NaN,n,NaN,sg,mann,4,"['n', 'm', 'sg']",222707.0
4,0.800000,1362082044,5963,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,8,6,5,...,f,NaN,NaN,n,NaN,sg,frau,4,"['n', 'f', 'sg']",143725.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12509996,0.800000,1363104897,368,u:i5D8,en,it,d5efc552aaea3109eb5388aa1ec8673d,6,4,5,...,NaN,def,NaN,det,NaN,sp,the,3,"['det', 'def', 'sp']",22761659.0
12509997,0.800000,1363104897,368,u:i5D8,en,it,a826c47947d68549fa81e19cafa57ba0,4,4,5,...,NaN,NaN,pres,vblex,NaN,NaN,eat,3,"['vblex', 'pres']",197654.0
12509998,1.000000,1363104897,368,u:i5D8,en,it,5e29d77697d23070a1fb92eb6c90e9b6,4,4,4,...,NaN,NaN,NaN,n,NaN,sg,bread,5,"['n', 'sg']",24828.0
12509999,0.600000,1363104897,368,u:i5D8,en,it,cdfecc9247566d40bb964a218c54c783,3,2,5,...,NaN,NaN,pres,vblex,NaN,NaN,drink,5,"['vblex', 'pres']",182580.0


In [35]:
df_2 = df_1.merge(df_users, on = ['user_id', 'lang_combination'], how='inner')

In [36]:
df_2

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,history_seen,history_correct,session_seen,...,person,number,word,word_len,tags_list,SUBTLEX,avg_user_p_recall,avg_delta,std_delta,avg_h_recall
0,1.000000,1362082032,444407,u:dDwF,es,en,73eecb492ca758ddab5371cf7b5cca32,3,3,1,...,NaN,NaN,bajo,4,['pr'],111241.0,0.885194,2.475405e+06,2.879771e+06,0.954897
1,1.000000,1362082044,5963,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,8,6,6,...,p3,sg,lernt,5,"['vblex', 'pri', 'p3', 'sg']",3391.0,0.614120,3.104417e+03,2.977079e+03,0.890225
2,0.750000,1362082044,5963,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,6,5,4,...,NaN,sg,die,3,"['det', 'def', 'f', 'sg']",2484854.0,0.614120,3.104417e+03,2.977079e+03,0.890225
3,0.888889,1362082044,5963,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,6,5,9,...,NaN,sg,mann,4,"['n', 'm', 'sg']",222707.0,0.614120,3.104417e+03,2.977079e+03,0.890225
4,0.800000,1362082044,5963,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,8,6,5,...,NaN,sg,frau,4,"['n', 'f', 'sg']",143725.0,0.614120,3.104417e+03,2.977079e+03,0.890225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12509996,0.800000,1363104897,368,u:i5D8,en,it,d5efc552aaea3109eb5388aa1ec8673d,6,4,5,...,NaN,sp,the,3,"['det', 'def', 'sp']",22761659.0,0.948715,5.391088e+04,6.550006e+04,0.892502
12509997,0.800000,1363104897,368,u:i5D8,en,it,a826c47947d68549fa81e19cafa57ba0,4,4,5,...,NaN,NaN,eat,3,"['vblex', 'pres']",197654.0,0.948715,5.391088e+04,6.550006e+04,0.892502
12509998,1.000000,1363104897,368,u:i5D8,en,it,5e29d77697d23070a1fb92eb6c90e9b6,4,4,4,...,NaN,sg,bread,5,"['n', 'sg']",24828.0,0.948715,5.391088e+04,6.550006e+04,0.892502
12509999,0.600000,1363104897,368,u:i5D8,en,it,cdfecc9247566d40bb964a218c54c783,3,2,5,...,NaN,NaN,drink,5,"['vblex', 'pres']",182580.0,0.948715,5.391088e+04,6.550006e+04,0.892502


In [37]:
df_2.columns

Index(['p_recall', 'timestamp', 'delta', 'user_id', 'learning_language',
       'ui_language', 'lexeme_id', 'history_seen', 'history_correct',
       'session_seen', 'session_correct', 'h_recall', 'lang_combination',
       'gender', 'def', 'tense', 'POS', 'person', 'number', 'word', 'word_len',
       'tags_list', 'SUBTLEX', 'avg_user_p_recall', 'avg_delta', 'std_delta',
       'avg_h_recall'],
      dtype='object')

In [38]:
### PREPARE DATA FOR ML 
dff = df_2.drop(columns=['timestamp', 'lexeme_id', 'word', 'user_id'])

In [39]:
dff.columns

Index(['p_recall', 'delta', 'learning_language', 'ui_language', 'history_seen',
       'history_correct', 'session_seen', 'session_correct', 'h_recall',
       'lang_combination', 'gender', 'def', 'tense', 'POS', 'person', 'number',
       'word_len', 'tags_list', 'SUBTLEX', 'avg_user_p_recall', 'avg_delta',
       'std_delta', 'avg_h_recall'],
      dtype='object')

In [53]:
categorical_cols = dff.select_dtypes(include=['object']).columns.tolist()

In [65]:
# Two approaches to handle tags 
# 1) Store tags as list, but delete rows where there are less than x occurences 
dff_1 = dff.copy()
tag_counts = dff_1['tags_list'].value_counts()
rare_threshold = 1000
dff_1['tags_list'] = dff_1['tags_list'].apply(lambda x: x if tag_counts[x] > rare_threshold else 'rare')

dff_1.drop(columns=['POS', 'person', 'number', 'gender', 'tense', 'def'], inplace=True)

In [74]:
# 2) Store each tag as column, replace NaN values with a placeholder for categorical columns
dff_2 = dff.copy()
dff_2[categorical_cols] = dff_2[categorical_cols].fillna('missing')
dff_2.drop(columns=['tags_list'], inplace=True)

In [60]:
# dff['gender'].value_counts(normalize=True)
# dff['tense'].value_counts(normalize=True)
# dff['person'].value_counts(normalize=True)
# dff['POS'].value_counts(normalize=True)
# dff['number'].value_counts(normalize=True)


number
sg      0.588309
Null    0.203243
pl      0.175885
sp      0.032563
Name: proportion, dtype: float64

In [76]:
dff_1

,p_recall,delta,learning_language,ui_language,history_seen,history_correct,session_seen,session_correct,h_recall,lang_combination,word_len,tags_list,SUBTLEX,avg_user_p_recall,avg_delta,std_delta,avg_h_recall
0,1.000000,444407,es,en,3,3,1,1,1.000000,en-es,4,['pr'],111241.0,0.885194,2.475405e+06,2.879771e+06,0.954897
1,1.000000,5963,de,en,8,6,6,6,0.750000,en-de,5,"['vblex', 'pri', 'p3', 'sg']",3391.0,0.614120,3.104417e+03,2.977079e+03,0.890225
2,0.750000,5963,de,en,6,5,4,3,0.833333,en-de,3,"['det', 'def', 'f', 'sg']",2484854.0,0.614120,3.104417e+03,2.977079e+03,0.890225
3,0.888889,5963,de,en,6,5,9,8,0.833333,en-de,4,"['n', 'm', 'sg']",222707.0,0.614120,3.104417e+03,2.977079e+03,0.890225
4,0.800000,5963,de,en,8,6,5,4,0.750000,en-de,4,"['n', 'f', 'sg']",143725.0,0.614120,3.104417e+03,2.977079e+03,0.890225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12509996,0.800000,368,en,it,6,4,5,4,0.666667,it-en,3,"['det', 'def', 'sp']",22761659.0,0.948715,5.391088e+04,6.550006e+04,0.892502
12509997,0.800000,368,en,it,4,4,5,4,1.000000,it-en,3,"['vblex', 'pres']",197654.0,0.948715,5.391088e+04,6.550006e+04,0.892502
12509998,1.000000,368,en,it,4,4,4,4,1.000000,it-en,5,"['n', 'sg']",24828.0,0.948715,5.391088e+04,6.550006e+04,0.892502
12509999,0.600000,368,en,it,3,2,5,3,0.666667,it-en,5,"['vblex', 'pres']",182580.0,0.948715,5.391088e+04,6.550006e+04,0.892502


In [80]:
# One-hot encode categorical variables
categorical_cols = dff_1.select_dtypes(include=['object']).columns.tolist()
ohe = OneHotEncoder(sparse_output=False)
ohe_data = ohe.fit_transform(dff_1[categorical_cols])
# Drop the original categorical columns
# df_encoded = df_encoded.drop(categorical_columns, axis=1)

In [79]:
ohe_data

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 50040004 stored elements and shape (12510001, 177)>

In [81]:
ohe_df = pd.DataFrame(ohe_data, columns=ohe.get_feature_names_out(categorical_cols))
df_encoded = pd.concat([dff_1.select_dtypes(exclude='O'), ohe_df], axis=1)


In [82]:
df_encoded

,p_recall,delta,history_seen,history_correct,session_seen,session_correct,h_recall,word_len,SUBTLEX,avg_user_p_recall,...,"tags_list_['vbser', 'pres']","tags_list_['vbser', 'pri', 'p1', 'pl']","tags_list_['vbser', 'pri', 'p1', 'sg']","tags_list_['vbser', 'pri', 'p2', 'pl']","tags_list_['vbser', 'pri', 'p2', 'sg']","tags_list_['vbser', 'pri', 'p3', 'pl']","tags_list_['vbser', 'pri', 'p3', 'sg']","tags_list_['vbser', 'pri']",tags_list_[],tags_list_rare
0,1.000000,444407,3,3,1,1,1.000000,4,111241.0,0.885194,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,5963,8,6,6,6,0.750000,5,3391.0,0.614120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.750000,5963,6,5,4,3,0.833333,3,2484854.0,0.614120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.888889,5963,6,5,9,8,0.833333,4,222707.0,0.614120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.800000,5963,8,6,5,4,0.750000,4,143725.0,0.614120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12509996,0.800000,368,6,4,5,4,0.666667,3,22761659.0,0.948715,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12509997,0.800000,368,4,4,5,4,1.000000,3,197654.0,0.948715,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12509998,1.000000,368,4,4,4,4,1.000000,5,24828.0,0.948715,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12509999,0.600000,368,3,2,5,3,0.666667,5,182580.0,0.948715,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.preprocessing import StandardScaler
numerical_cols = ['word_len', 'SUBTLEX', 'avg_delta']
scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

X = df_encoded.drop('p_recall', axis=1)  # Features
y = df_encoded['p_recall'] 